In [13]:
'''
2D interpolating function that takes in continuous water content and wavelength values. returns transmission. 
Wavelength range limited to 600 nm (inclusive) to 2400 nm (non-inclusive). 
Water content values limited to 1 to 5 mm. 
these values are dependent on the SKYCALC files read in, could be adjusted with different data.
Can take in a long list of wavelengths, which will be necessary for the spectra_transmission function.

Wavelength input can be array, water content must be value
'''

def interpolate_wave(water_content, wavelength):
    water_content = [water_content]
    wave_array = np.array(d["data1.0_pd"]["col1"])
    interp = rgi((water_array, wave_array), flux_array)
    result = list(product(water_content, wavelength))
    return interp(result)


In [15]:
'''
2D interpolating function that takes in continuous water content and wavelength values. returns transmission. 
Wavelength range limited to 600 nm (inclusive) to 2400 nm (non-inclusive). 
Water content values limited to 1 to 5 mm. 
these values are dependent on the SKYCALC files read in, could be adjusted with different data.
Can take in a long list of wavelengths, which will be necessary for the spectra_transmission function.

Water Content input can be array, wavelength content must be value
'''

def interpolate_water(water_content, wavelength):
    #water_content = [water_content]
    wavelength = [wavelength]
    wave_array = np.array(d["data1.0_pd"]["col1"])
    interp = rgi((water_array, wave_array), flux_array)
    result = list(product(water_content, wavelength))
    #print(result)
    return interp(result)

In [23]:
def division_noise(target_array, ref_array):
    new_array1 = []
    bin_size_array = [target_array[1][1]-target_array[1][0]]
    #new_array2
    i=0
    for n in target_array[0][0]:
        bin_size = target_array[1][i+1]-target_array[1][i]
        bin_size_array.append(bin_size)
        #print(i)
        target_array_i = np.array(target_array[0][:,i])
        #print(target_array_i)
        ref_array_i = np.array(ref_array[0][:,i])
        target_noise = photon_noise(target_array_i, bin_size)
        #print(target_noise)
        ref_noise = photon_noise(ref_array_i, bin_size)
        noisy_target_i = target_noise*target_array_i
        noisy_ref_i = ref_noise*ref_array_i
        #print(ref_array_i)
        divided_i = noisy_target_i/noisy_ref_i
        #new_array1 = np.append(new_array1, divided_i)
        new_array1.append(divided_i)
        #print(divided_i)
        i+=1
        #print(i)
    return new_array1

#cold_star_abs1_test3[:,0]


In [20]:
def division(target_array, ref_array):
    new_array1 = []
    bin_size_array = [target_array[1]-target_array[0]]
    #new_array2
    i=0
    for n in target_array[0]:
        bin_size = target_array[i+1]-target_array[i]
        bin_size_array.append(bin_size)
        #print(i)
        target_array_i = np.array(target_array[:,i])
        #print(target_array_i)
        ref_array_i = np.array(ref_array[:,i])
        #target_noise = photon_noise(target_array_i)
        #ref_noise = photon_noise(ref_array_i)
        #noisy_target_i = target_noise*target_array_i
        #noisy_ref_i = ref_noise*ref_array_i
        #print(ref_array_i)
        divided_i = target_array_i/ref_array_i
        #new_array1 = np.append(new_array1, divided_i)
        new_array1.append(divided_i)
        #print(divided_i)
        i+=1
        #print(i)
    return new_array1

#cold_star_abs1_test3[:,0]


In [21]:
def stacking(spectrum_array):
    bin_array = spectrum_array[0][1]
    spectrum_array = np.array(spectrum_array, dtype = object)
    spectrum_array = np.array(spectrum_array[:,0])
    spectrum_array = np.stack(spectrum_array)
    #print(spectrum_array)
    return spectrum_array, bin_array

In [22]:
def diff_spec(target_temp, ref_temp, target_pwv, ref_pwv, wave_in, wave_out, res=200, exp_time = 3600, throughput = .4):
    target_spec = spectrum(target_temp)
    ref_spec = spectrum(ref_temp)
    target_binned = binned_flux_pwv(target_spec, wave_in, wave_out, res)
    ref_binned = binned_flux_pwv(ref_spec, wave_in, wave_out, res)
    target_stacked = stacking(target_binned)
    ref_stacked = stacking(ref_binned)
    diff = division_noise(target_stacked, ref_stacked)
    
    #print(diff[0])
    
    normalized_array = []
    #photon_noise_array = 
    
    i=0
    for n in diff:
        '''
        input to photon noise needs to be actual flux value, not relative value after division. Needs to be multiplied into
        both target and ref fluxes?
        '''
        #photon_noise_i = photon_noise(diff[i], .5, exp_time, throughput)
        #print(photon_noise_i)
        #noisy_diff = diff[i]*photon_noise_i
        diff_mean = np.mean(diff[i])
        normalized = (diff[i]-diff_mean)/diff_mean
        normalized_array.append(normalized)
        #j=0
        #for n in normalized:
            #photon_noise(normalized[j], .5, exp_time, throughput
        #print(diff_mean)
        i+=1
    #diff_mean = np.mean(diff)
    #normalized = (diff-diff_mean)/diff_mean
    
    return normalized_array

In [26]:
def diff_spec_v2(target_temp, ref_temp, target_pwv, ref_pwv, wave_in, wave_out, mag1, mag2, res=200, exp_time = 3600, throughput = .4):
    target_spec = spectrum(target_temp, mag1)
    ref_spec = spectrum(ref_temp, mag2)
    target_binned = binned_flux_pwv_v2(target_spec, target_pwv, wave_in, wave_out, res)
    ref_binned = binned_flux_pwv_v2(ref_spec, ref_pwv, wave_in, wave_out, res)
    target_stacked = stacking(target_binned)
    ref_stacked = stacking(ref_binned)
    diff = division_noise(target_stacked, ref_stacked)
    
    #print(diff[0])
    
    normalized_array = []
    #photon_noise_array = 
    
    i=0
    for n in diff:
        '''
        input to photon noise needs to be actual flux value, not relative value after division. Needs to be multiplied into
        both target and ref fluxes?
        '''
        #photon_noise_i = photon_noise(diff[i], .5, exp_time, throughput)
        #print(photon_noise_i)
        #noisy_diff = diff[i]*photon_noise_i
        diff_mean = np.mean(diff[i])
        normalized = (diff[i]-diff_mean)/diff_mean
        normalized_array.append(normalized)
        #j=0
        #for n in normalized:
            #photon_noise(normalized[j], .5, exp_time, throughput
        #print(diff_mean)
        i+=1
    #diff_mean = np.mean(diff)
    #normalized = (diff-diff_mean)/diff_mean
    
    return normalized_array

In [ ]:
def generate_spectra(target_temp, ref_temp, mag1, mag2):
    target_spec = spectrum(target_temp, mag1)
    ref_spec = spectrum(ref_temp, mag2)
    '''
    target_binned = binned_flux_pwv_v2(target_spec, target_pwv, wave_in, wave_out, res)
    ref_binned = binned_flux_pwv_v2(ref_spec, ref_pwv, wave_in, wave_out, res)
    target_stacked = stacking(target_binned)
    ref_stacked = stacking(ref_binned)
    diff = division_noise(target_stacked, ref_stacked)
    
    #print(diff[0])
    
    normalized_array = []
    #photon_noise_array = 
    
    i=0
    for n in diff:
        
        input to photon noise needs to be actual flux value, not relative value after division. Needs to be multiplied into
        both target and ref fluxes?
        
        #photon_noise_i = photon_noise(diff[i], .5, exp_time, throughput)
        #print(photon_noise_i)
        #noisy_diff = diff[i]*photon_noise_i
        diff_mean = np.mean(diff[i])
        normalized = (diff[i]-diff_mean)/diff_mean
        normalized_array.append(normalized)
        #j=0
        #for n in normalized:
            #photon_noise(normalized[j], .5, exp_time, throughput
        #print(diff_mean)
        i+=1
    #diff_mean = np.mean(diff)
    #normalized = (diff-diff_mean)/diff_mean
    '''
    return target_spec, ref_spec

In [2]:

def bin_divide(target_spec, ref_spec, target_pwv, ref_pwv, wave_in, wave_out, atm, rp, res=200, exp_time = 3600, throughput = .4):
    target_binned = binned_flux_pwv_v2(target_spec, target_pwv, wave_in, wave_out, res)
    #wave_array = target_binned_all[1]
    ref_binned = binned_flux_pwv_v2(ref_spec, ref_pwv, wave_in, wave_out, res)
    target_stacked = stacking(target_binned)
    ref_stacked = stacking(ref_binned)
    diff = division_noise_transit(target_stacked, ref_stacked, atm, rp)
    return diff


In [ ]:
def normalize(diff):
    '''
    target_spec = spectrum(target_temp, mag1)
    ref_spec = spectrum(ref_temp, mag2)
    
    target_binned = binned_flux_pwv_v2(target_spec, target_pwv, wave_in, wave_out, res)
    ref_binned = binned_flux_pwv_v2(ref_spec, ref_pwv, wave_in, wave_out, res)
    target_stacked = stacking(target_binned)
    ref_stacked = stacking(ref_binned)
    diff = division_noise(target_stacked, ref_stacked)
    
    #print(diff[0])
    '''
    normalized_array = []
    #photon_noise_array = 
    
    i=0
    for n in diff:
        '''
        input to photon noise needs to be actual flux value, not relative value after division. Needs to be multiplied into
        both target and ref fluxes?
        '''
        #photon_noise_i = photon_noise(diff[i], .5, exp_time, throughput)
        #print(photon_noise_i)
        #noisy_diff = diff[i]*photon_noise_i
        diff_mean = (np.mean(diff[i][0:150])+np.mean(diff[i][390:540]))/2
        normalized = (diff[i]-diff_mean)/diff_mean
        normalized_array.append(normalized)
        #j=0
        #for n in normalized:
            #photon_noise(normalized[j], .5, exp_time, throughput
        #print(diff_mean)
        i+=1
    #diff_mean = np.mean(diff)
    #normalized = (diff-diff_mean)/diff_mean
    
    
    return normalized_array

In [ ]:
def division_noise_transit(target_array, ref_array, atm, params):
    new_array1 = []
    bin_size_array = [target_array[1][1]-target_array[1][0]]
    
    t = np.linspace(-0.5, 0.5, 540)
    #print("right function!")
    bulk_transit_depth = params.rp**2
    print("bulk_transit_depth: "+str(bulk_transit_depth))
    print("bulk_rp: "+str(params.rp))
    #new_array2
    i=0
    for n in target_array[0][0]:
        if i >= len(atm["col5"]):
            break
        
        bin_size = target_array[1][i+1]-target_array[1][i]
        #print("bin size: "+str(bin_size))
        bin_size_array.append(bin_size)
        target_array_i = np.array(target_array[0][:,i])
        #print(target_array_i[50:60])
        #print("original target array"+str(target_array_i[0:10]))
        
        #adding transit
        atm_transit_i = atm["col5"][i]/10**6
        #print("atm transit: "+ str(atm_transit_i))
        #print("atm depth"+str(atm_transit_i))
        
        total_transit_depth = bulk_transit_depth + atm_transit_i
        #print("total transit depth: "+ str(total_transit_depth))
        rp_i = np.sqrt(total_transit_depth)
        #print("rp i : "+ str(rp_i))
        params.rp = rp_i
        #print("edited param : "+ str(params.rp))
        #print("rp"+str(params.rp))
        model = batman.TransitModel(params, t)
        transit = model.light_curve(params)
        #print("transit model"+str(transit[500:510]))
        plt.plot(transit)
        
        #flux = stellar_flux_value*transmission
        print("transit length " + str(len(transit)))
        print("target array length " + str(len(target_array_i)))
        transit_flux = transit*target_array_i

        plt.plot(transit_flux)
        #print("flux with transit"+str(transit_flux[0:10]))
        
        
        ref_array_i = np.array(ref_array[0][:,i])
        #print(ref_array_i[50:60])
        target_noise = photon_noise(transit_flux, bin_size)
        #print("target_noise: "+str(target_noise))
        ref_noise = photon_noise(ref_array_i, bin_size)
        #print("ref_noise: "+str(ref_noise))
        noisy_target_i = target_noise*transit_flux
        #plt.plot(noisy_target_i)
        #print("target_noisy flux: "+str(noisy_target_i))
        noisy_ref_i = ref_noise*ref_array_i
        #print("ref_noisy flux: "+str(noisy_ref_i))
        divided_i = noisy_target_i/noisy_ref_i
        #plt.plot(divided_i)
        #print("div_noisy flux: "+str(divided_i))
        new_array1.append(divided_i)
        i+=1
        #if i == 3:
            #break
    return new_array1

#cold_star_abs1_test3[:,0]
